In [1]:
import pandas as pd
from somda_project.s3_funcs import create_minio_client,retrieve_file
from dotenv import load_dotenv
load_dotenv()

In [ ]:
endpoint = os.environ.get("BUCKET_ENDPOINT")
bucket_id = os.environ.get("BUCKET_ID")
access_key = os.environ.get("BUCKET_ACCESS_KEY_ID")
secret_key = os.environ.get("BUCKET_SECRET_KEY")
region = os.environ.get("BUCKET_REGION")
client = create_minio_client(endpoint, access_key, secret_key, region)

In [ ]:
csv = retrieve_file(client, f"merged_election_pages.csv", bucket_id)